# Bitcoin Logistic Regression Pipeline

此 notebook 示範如何下載 Binance BTC/USDT 的歷史資料、計算技術指標特徵，並以邏輯迴歸模型預測下一期價格是否上漲，同時進行簡易回測。

## 1. 下載資料

首先設定資料來源與讀取函式。如果本地 `data/` 目錄尚未有資料，會自動呼叫 `fetch_data.py` 從 Binance API 抓取 1 小時 K 線。

In [ ]:
import datetime as dt
from pathlib import Path
import subprocess

import numpy as np
import pandas as pd

DATA_PATH = Path("data/btcusdt_1h.csv")

if not DATA_PATH.exists():
    print("Data file not found, downloading from Binance...")
    subprocess.run([
        "python",
        "fetch_data.py",
        "--symbol",
        "BTCUSDT",
        "--interval",
        "1h",
        "--start",
        (dt.datetime.utcnow() - dt.timedelta(days=180)).replace(minute=0, second=0, microsecond=0).isoformat(),
        "--end",
        dt.datetime.utcnow().replace(minute=0, second=0, microsecond=0).isoformat(),
        "--output",
        str(DATA_PATH),
    ], check=True)
else:
    print(f"Using cached data: {DATA_PATH}")


## 2. 載入並整理資料

轉換時間欄位並計算報酬率，快速檢查資料狀況。

In [ ]:
df = pd.read_csv(DATA_PATH)
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
df = df.sort_values('open_time').reset_index(drop=True)
df['return'] = df['close'].pct_change()
df.tail()


## 3. 建立技術指標特徵

包含下列指標：

1. **Momentum**：`close` 與 3 期前的差值。
2. **SMA**：10 期簡單移動平均。
3. **Volatility**：10 期報酬率的標準差。
4. **RSI**：14 期相對強弱指標。
5. **Volume**：以 z-score 正規化的成交量。

In [ ]:
window_sma = 10
window_momentum = 3
window_vol = 10
window_rsi = 14

df['momentum'] = df['close'] - df['close'].shift(window_momentum)
df['sma'] = df['close'].rolling(window_sma).mean()
df['volatility'] = df['return'].rolling(window_vol).std()

# RSI 計算
change = df['close'].diff()
gain = change.clip(lower=0)
loss = -change.clip(upper=0)
avg_gain = gain.rolling(window_rsi).mean()
avg_loss = loss.rolling(window_rsi).mean()
rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))
df['rsi'] = rsi
df['volume_z'] = (df['volume'] - df['volume'].rolling(window_vol).mean()) / df['volume'].rolling(window_vol).std()

feature_cols = ['momentum', 'sma', 'volatility', 'rsi', 'volume_z']
df_features = df.dropna(subset=feature_cols).copy()
df_features['target'] = (df_features['close'].shift(-1) > df_features['close']).astype(int)
df_features = df_features.dropna(subset=['target'])
feature_cols, df_features.head()


## 4. 切分資料與訓練模型

使用時間序列的 70% 做訓練，其餘做測試，並以 scikit-learn 的 `LogisticRegression` 進行模型訓練。

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

split_idx = int(len(df_features) * 0.7)
train = df_features.iloc[:split_idx]
test = df_features.iloc[split_idx:]

X_train, y_train = train[feature_cols], train['target']
X_test, y_test = test[feature_cols], test['target']

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


## 5. 回測策略

策略假設：
- 若預測下一期價格上漲機率大於 0.5 則持有多頭，否則持有空頭。
- 回測以 `close` 價格的報酬率作為標的。

In [ ]:
import matplotlib.pyplot as plt

backtest = test.copy()
backtest = backtest.assign(
    prob=y_proba,
    position=np.where(y_proba > 0.5, 1, -1)
)
backtest['strategy_return'] = backtest['position'] * backtest['return'].shift(-1)
backtest['buy_hold'] = backtest['return'].shift(-1)
backtest[['strategy_return', 'buy_hold']] = backtest[['strategy_return', 'buy_hold']].fillna(0)

backtest['strategy_cum'] = (1 + backtest['strategy_return']).cumprod()
backtest['buy_hold_cum'] = (1 + backtest['buy_hold']).cumprod()

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(backtest['open_time'], backtest['strategy_cum'], label='Strategy (LR)')
ax.plot(backtest['open_time'], backtest['buy_hold_cum'], label='Buy & Hold')
ax.set_title('Backtest Cumulative Returns')
ax.set_ylabel('Equity Curve')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 6. 結論

以少量技術指標建立的簡易邏輯迴歸模型能提供價格方向的初步判斷，但仍需考慮特徵工程與風險控管。接下來可嘗試更多指標、不同時間週期或加入交易成本假設。